# 核函数

In [86]:
import numpy as np 

def kernel_trans(X, A, kTup):
    m,n = np.shape(X)
    K = np.mat(np.zeros((m,1)))
    if kTup[0]=='lin': K = X * A.T   #linear kernel
    elif kTup[0]=='rbf':
        for j in range(m):
            deltaRow = X[j,:] - A
            K[j] = deltaRow*deltaRow.T
        K = np.exp(K/(-1*kTup[1]**2)) #divide in NumPy is element-wise not matrix like Matlab
    else: raise NameError('Houston We Have a Problem -- \
    That Kernel is not recognized')
    return K

class optStruct:
    def __init__(self, datamat, classlabels, C, toler, kTup):
        self.X = datamat 
        self.label_mat = classlabels
        self.C = C 
        self.tol = toler 
        self.m = np.shape(datamat)[0]
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        self.ecache = np.mat(np.zeros((self.m, 2)))
        self.K = np.mat(np.zeros((self.m, self.m)))
        for i in range(self.m):
            self.K[:,i] = kernel_trans(self.X, self.X[i,:], kTup)

In [32]:
def load_dataset(filename):
    data_mat = []; label_mat = []
    with open(filename) as fr:
        for line in fr.readlines():
            line_arr = line.strip().split('\t')
            data_mat.append([float(line_arr[0]), float(line_arr[1])])
            label_mat.append(float(line_arr[2]))
    return data_mat, label_mat

In [94]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.label_mat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.label_mat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = select_j(i, oS, Ei)
        alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy()
        if (oS.label_mat[i] != oS.label_mat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H: return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j] #changed for kernel
        if eta >= 0: return 0
        oS.alphas[j] -= oS.label_mat[j]*(Ei - Ej)/eta
        oS.alphas[j] = clip_alpha(oS.alphas[j],H,L)
        updateEk(oS, j) #added this for the Ecache
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): return 0
        oS.alphas[i] += oS.label_mat[j]*oS.label_mat[i]*(alphaJold - oS.alphas[j])#update i by the same amount as j
        updateEk(oS, i) #added this for the Ecache                    #the update is in the oppostie direction
        b1 = oS.b - Ei- oS.label_mat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i] - oS.label_mat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2 = oS.b - Ej- oS.label_mat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]- oS.label_mat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2)/2.0
        return 1
    else: return 0

In [95]:
def smo_p(data_mat, classlabels, C, toler, max_iter, kTup):
    """完整SMO算法中的外循环代码"""
    ds = optStruct(np.mat(data_mat), np.mat(classlabels).transpose(), C, toler, kTup)
    num = 0
    entire_set = True 
    alpha_pairs_changed = 0
    while (num < max_iter) and ((alpha_pairs_changed > 0) or (entire_set)):
        alpha_pairs_changed = 0
        if entire_set:  # 遍历所有值
            for i in range(ds.m):
                alpha_pairs_changed += innerL(i, ds)
            # print("Fullset, iter: %d i: %d, pairs changed %d" %\
            #     (num, i, alpha_pairs_changed))
            num += 1
        else:       # 遍历非边界值
            non_bounds = np.nonzero((np.array(ds.alphas) > 0) * (np.array(ds.alphas) < C))[0]
            for i in non_bounds:
                alpha_pairs_changed += innerL(i, ds)
                # print("non-bound, iter: %d i: %d, pairs changed %d" %\
                #     (num, i, alpha_pairs_changed))
                num += 1
        if entire_set:
            entire_set = False 
        elif alpha_pairs_changed == 0:
            entire_set = True 
        # print("iteration number: %d" % num)
    return ds.b, ds.alphas

In [96]:
def calcEk(oS, k):
    fXk = float(np.multiply(oS.alphas,oS.label_mat).T*oS.K[:,k] + oS.b)
    Ek = fXk - float(oS.label_mat[k])
    return Ek

def updateEk(ds, k):
    Ek = calcEk(ds, k)
    ds.ecache[k] = [1, Ek]

def select_jrand(i, m):
    # 随机选择另一个优化的j,不等于i
    j = i 
    while j == i:
        j = np.random.randint(0, m)
    return j

def select_j(i, ds, Ei):
    max_k = -1
    max_deltaE = 0
    Ej = 0  # 内循环中的启发式方法
    ds.ecache[i] = [1, Ei]
    valid_Ecache_lst = np.nonzero(np.array(ds.ecache[:, 0]))[0]
    if (len(valid_Ecache_lst) > 1):
        # >1说明至之前计算过别的E，选择最大的进行优化
        for k in valid_Ecache_lst:
            if k == i:
                continue
            Ek = calcEk(ds, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > max_deltaE):
                max_k = k
                max_deltaE = deltaE 
        return max_k, Ej 
    else:
        # 第一次循环就直接随机选择一个j
        j = select_jrand(i, ds.m)
        Ej = calcEk(ds, j)
    return j, Ej 

def clip_alpha(aj, H, L):
    aj = max(aj, L)
    aj = min(aj, H)
    return aj

In [97]:
def testRbf(k1=1.3):
    data_arr, label_arr = load_dataset('testSetRBF.txt')
    b, alphas = smo_p(data_arr, label_arr, 200, 0.0001, 10000, ('rbf', k1))
    datamat = np.mat(data_arr)
    labelmat = np.mat(label_arr).transpose()
    svInd = np.nonzero(alphas.A>0)[0]
    sVs = datamat[svInd]
    labelSV = labelmat[svInd]
    print("There are %d support vectors" % np.shape(sVs)[0])
    m,n = np.shape(datamat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernel_trans(sVs, datamat[i,:], ('rbf', k1))
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b 
        if np.sign(predict) != np.sign(label_arr[i]):
            errorCount += 1
    print("The training error rate is %f" % (float(errorCount)/m))

    data_arr, label_arr = load_dataset('testSetRBF2.txt')
    errorCount = 0
    datamat = np.mat(data_arr)
    labelmat = np.mat(label_arr).transpose()
    m,n = np.shape(datamat)
    for i in range(m):
        kernelEval = kernel_trans(sVs, datamat[i,:], ('rbf', k1))
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b 
        if np.sign(predict) != np.sign(label_arr[i]):
            errorCount += 1
    print("The testing error rate is %f" % (float(errorCount)/m))

In [102]:
testRbf()

There are 38 support vectors
The training error rate is 0.100000
The testing error rate is 0.150000


# 手写识别问题

In [ ]:
def img2vector(filename):
    returnVect = np.zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect

def loadImages(dirName):
    from os import listdir
    hwLabels = []
    trainingFileList = listdir(dirName)           #load the training set
    m = len(trainingFileList)
    trainingMat = np.zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]     #take off .txt
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9: hwLabels.append(-1)
        else: hwLabels.append(1)
        trainingMat[i,:] = img2vector('%s/%s' % (dirName, fileNameStr))
    return trainingMat, hwLabels  

def testDigits(kTup=('rbf', 10)):
    dataArr,labelArr = loadImages('trainingDigits')
    b,alphas = smo_p(dataArr, labelArr, 200, 0.0001, 10000, kTup)
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).transpose()
    svInd = np.nonzero(alphas.A>0)[0]
    sVs=datMat[svInd] 
    labelSV = labelMat[svInd]
    print ("there are %d Support Vectors" % np.shape(sVs)[0])
    m,n = np.shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernel_trans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * np.multiply(labelSV,alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelArr[i]): errorCount += 1
    print ("the training error rate is: %f" % (float(errorCount)/m))
    
    dataArr,labelArr = loadImages('testDigits')
    errorCount = 0
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).transpose()
    m,n = np.shape(datMat)
    for i in range(m):
        kernelEval = kernel_trans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * np.multiply(labelSV,alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelArr[i]): errorCount += 1    
    print ("the test error rate is: %f" % (float(errorCount)/m))

In [99]:
testDigits()

there are 93 Support Vectors
the training error rate is: 0.000000
the test error rate is: 0.005376
